In [234]:
%config Completer.use_jedi = False
import pandas as pd
import re
import collections
from matplotlib import pyplot as plt
import seaborn as sns
import pymysql
import random
import json
import os
from hydra.experimental import compose, initialize
# initialize(config_path="./", strict=False)

**함수**

In [235]:
ptn_list = [
    re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE),
    re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE),
    re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$'),
    re.compile('[a-zA-Z가-힣]+ ?뉴스|뉴스+[a-zA-Z가-힣0-9]+ ?|[a-zA-Z가-힣]+newskr|[a-zA-Z가-힣]+ ?Copyrights'),
    re.compile('\s+Copyrights|\s+배포 ?금지'),
    re.compile('[a-zA-Z가-힣]+com|[a-zA-Z가-힣]+.kr'),
    re.compile('[가-힣]+ ?기자|[가-힣]+ ?신문|[가-힣]+투데이|[가-힣]+미디어|[가-힣]+ ?데일리|[가-힣]+ 콘텐츠 무단|[가-힣]+ ?전재|[가-힣]+배포 ?금지'),
    re.compile('데일리+[가-힣]'),
    re.compile(r'전재\s+변형'),
    re.compile('\s+배포 ?금지'),
    re.compile('[a-zA-Z가-힣]+.kr'),
    re.compile('/^[a-z0-9_+.-]+@([a-z0-9-]+\.)+[a-z0-9]{2,4}$/'),
    re.compile('[\r|\n]'),
    re.compile('\[[^)]*\]'),
    re.compile('[^ ㄱ-ㅣ가-힣A-Za-z0-9]'),
    re.compile('뉴스코리아|및|Copyright|저작권자|ZDNET A RED VENTURES COMPANY'),
    re.compile('\([^)]*\)'),
    re.compile('[-=+,#\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》·▲\n]'),
    re.compile(r"[\s]{2,}"),
    re.compile(r'/+[a-zA-Z가-힣]'),
]

def processing(content):
    
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\t)')
    result = str(content)
    for i, ptn in enumerate(ptn_list):
        result = ptn.sub('.' if i == 2 else "", result)
        result = pattern_bracket.sub('', result).strip()
    return result

In [236]:
def category_add(content):
    labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']    
    k_i_dict = {k: i for i, k in enumerate(labels)}
    
    category = []
    for key, value in k_i_dict.items():
        if str(key) in content:
            category.append(value)

    if not category:
        category=[29]
    
    return category

In [237]:
def categorytext_add(content):
    labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']    
    k_i_dict = {k: i for i, k in enumerate(labels)}
    
    category = ""
    
    for key, value in k_i_dict.items():
        if str(key) in content:
            if category:
                category += ","            
            category += str(value)
            
    if not category:
        category="29"
    
    return category

In [238]:
def label_modify(category):
    if len(category) > 3:
        random.shuffle(category)
        category = category[:3]

    return category

In [239]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

**Labeling**

In [242]:
cfg = compose("config.yaml")

conn = pymysql.connect(
    user=cfg.Connection.user,
    passwd=cfg.Connection.passwd,
    db=cfg.Connection.db,
    host=cfg.Connection.host,
    port=cfg.Connection.port,
    charset="utf8",
    use_unicode=True,
    )

cursor = conn.cursor(pymysql.cursors.DictCursor)

query="""select id, content from portal_news_scraper.portal_news where portal='daum' and kind not in ('200206', '200207') limit 500000;"""            

cursor.execute(
    query
)
data = pd.DataFrame(cursor.fetchall())

In [243]:
data['content'] = data.content.apply(processing) 

In [244]:
data['content'][0]

'서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤 제37기 정기 주주총회에 참석하고 있다SK텔레콤이 주주가치 제고를 위해 한국거래소에 자사주 869만주 소각을 반영한 변경상장을 완료했다고 14일 밝혔다 발행주식 총수의약 2조6000억원규모다 이로써 전날 기준 SK텔레콤의 발행주식 총수는 기존 8075만주에서 7206만주로 줄어들었다앞서 SK텔레콤은 지난 4일 올해 인적분할에 앞서 기업가치주주가치 제고를 위해 기존 자사주를 사실상 전량 소각한다고 발표한 바 있다 이번 자사주 소각으로 발행주식 총수가 감소하면서 기존 주주들의 지분율이 모두 상승했다SK텔레콤은 분할 후 기업가치가 올라갈 것으로 전망하는 증권업계 전반의 시각을 감안하면 기업 펀더멘털 변동없이 주식 수만 줄어든 상황이라며 자사주 소각 전보다 주식가치 상승 여력이 더 커졌다고 분석했다SK텔레콤은 발행주식 총수가 줄었음에도 전날 시가총액이 약 22조5000억원을 기록해 주주총회에서 기업구조인적분할 개편을 공식화했던 지난 3월25일 시가총액약 20조5000억원보다 10가량 늘었다고 설명했다 주가도 연초 대비 30 이상 상승했다고 덧붙였다한편 SK텔레콤은 상반기 내 이사회 의결을 거쳐 오는 10월 주주총회 11월 인적분할 법인의 재상장을 통해 인적분할 절차를 끝낼 계획이다'

In [245]:
print(data.content)

0         서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤...
1         네오위즈공동대표 문지수 김승철의 신작 게임 블레스 언리쉬드 PC에 대한 글로벌 이용...
2         삼성 디지털프라자 삼성대치본점 갤럭시 스튜디오에서 소비자들이 갤럭시 북 프로 시리즈...
3         김창룡 방송통신위원회 상임위원이 온라인 동영상 서비스OTT 웨이브를 방문해 국내 O...
4         NHN이 게임 부문 실적 개선으로 우상향 그래프를 그렸다 다만 게임 외 사업이 대부...
                                ...                        
499995    국내 코로나19 백신 1차 접종률이 40를 넘었습니다지난 2월 26일 국내에서 백신...
499996    직계가족 모임도 4인까지만 허용유흥시설 6종 집합금지 해제하고 영업시간 제한광주 동...
499997    통신선 복원 후 한미 외교장관 첫 통화대북 인도적 지원 미국 이해 확보한 듯3월 1...
499998    농협중앙회가 6일 625전쟁 70주년 사업에 기여한 공로로 국가보훈처로 부터 감사패...
499999    알래스카 케치캔에 정박중인 홀란드 아메리카 라인 크루즈 미국 알래스카에서 관광용 경...
Name: content, Length: 500000, dtype: object


In [246]:
data = data.dropna()

In [247]:
data.count()
#there is no data less than 50000

id         500000
content    500000
dtype: int64

In [248]:
data['category'] = data.content.apply(category_add)

In [249]:
(data['category']).value_counts()

[29]                     301418
[23]                      18705
[21]                      14737
[9]                       13586
[10]                      13188
                          ...  
[3, 9, 11, 16, 27]            1
[5, 16, 25, 27]               1
[3, 5, 9, 12, 21, 23]         1
[1, 9, 20, 27]                1
[3, 5, 21, 24]                1
Name: category, Length: 6198, dtype: int64

In [250]:
data.head()

,id,content,category
0,315136,서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤...,[19]
1,315207,네오위즈공동대표 문지수 김승철의 신작 게임 블레스 언리쉬드 PC에 대한 글로벌 이용...,"[4, 9]"
2,315210,삼성 디지털프라자 삼성대치본점 갤럭시 스튜디오에서 소비자들이 갤럭시 북 프로 시리즈...,[29]
3,315241,김창룡 방송통신위원회 상임위원이 온라인 동영상 서비스OTT 웨이브를 방문해 국내 O...,[10]
4,315410,NHN이 게임 부문 실적 개선으로 우상향 그래프를 그렸다 다만 게임 외 사업이 대부...,[29]


**count check**

In [14]:
# labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']
# k_i_dict = {k: i for i, k in enumerate(labels)}

# tolist = data.category.tolist()
# tolist = sum(tolist, [])
# for key, value in k_i_dict.items():
#     a = tolist.count(value)
#     print(key + " : " + str(a))

인공지능 : 9817
로봇 : 3867
스마트팜 : 469
에너지 : 15742
서버 : 3001
투자 : 44129
정부지원 : 406
증강현실 : 1259
이동수단 : 315
개발 : 59245
통신 : 44557
과학 : 19849
드론 : 2130
블록체인 : 3098
핀테크 : 1882
커머스 : 4891
여행 : 10169
미디어 : 9806
헬스케어 : 2979
의약 : 7373
식품 : 15140
교육 : 35178
직업 : 4911
경제 : 83120
광고 : 6793
제약 : 8440
O2O : 187
뷰티 : 2261
부동산 : 19396
etc : 224233


**drop etc**

In [251]:
data['categorytext'] = data.content.apply(categorytext_add)

In [252]:
data['categorytext'].head(20)

0               19
1              4,9
2               29
3               10
4               29
5     0,9,10,11,23
6               16
7               29
8               29
9         3,5,9,10
10              11
11              29
12             4,9
13             4,9
14              29
15              29
16              29
17              29
18          4,9,10
19              29
Name: categorytext, dtype: object

In [253]:
index = data['categorytext'].isin(['29']) & (data['id'] <= 1800000) 
data2 = data[index == False]

In [254]:
data2['categorytext'].head()

0              19
1             4,9
3              10
5    0,9,10,11,23
6              16
Name: categorytext, dtype: object

In [255]:
data2.head()

,id,content,category,categorytext
0,315136,서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤...,[19],19
1,315207,네오위즈공동대표 문지수 김승철의 신작 게임 블레스 언리쉬드 PC에 대한 글로벌 이용...,"[4, 9]","4,9"
3,315241,김창룡 방송통신위원회 상임위원이 온라인 동영상 서비스OTT 웨이브를 방문해 국내 O...,[10],10
5,315442,문재인 대통령이 14일 오전 청와대에서 임혜숙 과학기술정보통신부 장관에게 임명장을 ...,"[0, 9, 10, 11, 23]","0,9,10,11,23"
6,315534,미국 뉴욕 전경최소 모집정원 1명짜리 파격 해외 패키지여행 상품이 등장했다최소 15...,[16],16


In [256]:
data2.count()

id              198582
content         198582
category        198582
categorytext    198582
dtype: int64

In [257]:
data2.category

0                       [19]
1                     [4, 9]
3                       [10]
5         [0, 9, 10, 11, 23]
6                       [16]
                 ...        
499992                  [21]
499996              [16, 23]
499997                  [10]
499998                  [20]
499999              [10, 16]
Name: category, Length: 198582, dtype: object

In [258]:
labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']
k_i_dict = {k: i for i, k in enumerate(labels)}
#{'인공지능': 0,  '로봇': 1,  '스마트팜': 2,...}

tolist = data2.category.tolist()
tolist = sum(tolist, [])
#[16, 23, 23, 0, 9, ...]

for key, value in k_i_dict.items():
    a = tolist.count(value)
    #count the value
    #46 15 3 61 ...
    print(key + " : " + str(a))

인공지능 : 8524
로봇 : 3553
스마트팜 : 399
에너지 : 14282
서버 : 2697
투자 : 39903
정부지원 : 345
증강현실 : 1213
이동수단 : 298
개발 : 54296
통신 : 32716
과학 : 18810
드론 : 2211
블록체인 : 2608
핀테크 : 1609
커머스 : 3973
여행 : 10862
미디어 : 6309
헬스케어 : 2633
의약 : 7280
식품 : 13868
교육 : 32626
직업 : 4720
경제 : 51017
광고 : 6375
제약 : 8180
O2O : 192
뷰티 : 2322
부동산 : 18173
etc : 65


In [259]:
data2 = data2[['category', 'content']]

In [260]:
data2

,category,content
0,[19],서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤...
1,"[4, 9]",네오위즈공동대표 문지수 김승철의 신작 게임 블레스 언리쉬드 PC에 대한 글로벌 이용...
3,[10],김창룡 방송통신위원회 상임위원이 온라인 동영상 서비스OTT 웨이브를 방문해 국내 O...
5,"[0, 9, 10, 11, 23]",문재인 대통령이 14일 오전 청와대에서 임혜숙 과학기술정보통신부 장관에게 임명장을 ...
6,[16],미국 뉴욕 전경최소 모집정원 1명짜리 파격 해외 패키지여행 상품이 등장했다최소 15...
...,...,...
499992,[21],재판부는 또 A씨에게 약물중독 재활교육 프로그램 40시간 이수를 명령했다A씨는 지난...
499996,"[16, 23]",직계가족 모임도 4인까지만 허용유흥시설 6종 집합금지 해제하고 영업시간 제한광주 동...
499997,[10],통신선 복원 후 한미 외교장관 첫 통화대북 인도적 지원 미국 이해 확보한 듯3월 1...
499998,[20],농협중앙회가 6일 625전쟁 70주년 사업에 기여한 공로로 국가보훈처로 부터 감사패...


**label modify**

In [261]:
data['category2'] = data2.category.apply(label_modify)
data.head()

,id,content,category,categorytext,category2
0,315136,서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤...,[19],19,[19]
1,315207,네오위즈공동대표 문지수 김승철의 신작 게임 블레스 언리쉬드 PC에 대한 글로벌 이용...,"[4, 9]","4,9","[4, 9]"
2,315210,삼성 디지털프라자 삼성대치본점 갤럭시 스튜디오에서 소비자들이 갤럭시 북 프로 시리즈...,[29],29,NaN
3,315241,김창룡 방송통신위원회 상임위원이 온라인 동영상 서비스OTT 웨이브를 방문해 국내 O...,[10],10,[10]
4,315410,NHN이 게임 부문 실적 개선으로 우상향 그래프를 그렸다 다만 게임 외 사업이 대부...,[29],29,NaN


In [262]:
data.rename(columns={'category':'origin_category', 'category2':'category'}, inplace=True)

In [263]:
data = data[['category', 'content']]

In [264]:
data = data.dropna()

In [265]:
data.count()

category    198582
content     198582
dtype: int64

**data save**

In [266]:
df = pd.DataFrame(data)

In [267]:

df.to_csv('/home/jh/documents/datasets/news_category.csv', encoding='utf-8')


In [268]:
temp_dict = [
    {"content": row['content'].strip(), "category": row['category']}
    for _, row in df.iterrows()
]

temp_dict[0]

{'content': '서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤 제37기 정기 주주총회에 참석하고 있다SK텔레콤이 주주가치 제고를 위해 한국거래소에 자사주 869만주 소각을 반영한 변경상장을 완료했다고 14일 밝혔다 발행주식 총수의약 2조6000억원규모다 이로써 전날 기준 SK텔레콤의 발행주식 총수는 기존 8075만주에서 7206만주로 줄어들었다앞서 SK텔레콤은 지난 4일 올해 인적분할에 앞서 기업가치주주가치 제고를 위해 기존 자사주를 사실상 전량 소각한다고 발표한 바 있다 이번 자사주 소각으로 발행주식 총수가 감소하면서 기존 주주들의 지분율이 모두 상승했다SK텔레콤은 분할 후 기업가치가 올라갈 것으로 전망하는 증권업계 전반의 시각을 감안하면 기업 펀더멘털 변동없이 주식 수만 줄어든 상황이라며 자사주 소각 전보다 주식가치 상승 여력이 더 커졌다고 분석했다SK텔레콤은 발행주식 총수가 줄었음에도 전날 시가총액이 약 22조5000억원을 기록해 주주총회에서 기업구조인적분할 개편을 공식화했던 지난 3월25일 시가총액약 20조5000억원보다 10가량 늘었다고 설명했다 주가도 연초 대비 30 이상 상승했다고 덧붙였다한편 SK텔레콤은 상반기 내 이사회 의결을 거쳐 오는 10월 주주총회 11월 인적분할 법인의 재상장을 통해 인적분할 절차를 끝낼 계획이다',
 'category': [19]}

In [269]:

!rm -rf /workspace/data/git_category.json
dump_jsonl(temp_dict, '/home/jh/documents/datasets/git_category.json')

Wrote 198582 records to /home/jh/documents/datasets/git_category.json


In [270]:
df.head()

,category,content
0,[19],서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤...
1,"[4, 9]",네오위즈공동대표 문지수 김승철의 신작 게임 블레스 언리쉬드 PC에 대한 글로벌 이용...
3,[10],김창룡 방송통신위원회 상임위원이 온라인 동영상 서비스OTT 웨이브를 방문해 국내 O...
5,"[10, 0, 11]",문재인 대통령이 14일 오전 청와대에서 임혜숙 과학기술정보통신부 장관에게 임명장을 ...
6,[16],미국 뉴욕 전경최소 모집정원 1명짜리 파격 해외 패키지여행 상품이 등장했다최소 15...


In [271]:
df.count()

category    198582
content     198582
dtype: int64